In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys, os
utils_path = Path("C:/Users/shaur/Desktop/Learnings/KG_Tests/senior_living_dm/src/utils")
sys.path.append(str(utils_path))

In [41]:
import json
import yaml

import re

from pdf2image import convert_from_path
from PIL import Image

from prompt_division_to_entities import call_openai_for_entity_division

## Define Paths

In [12]:
base_path = Path("C:/Users/shaur/Desktop/Learnings/KG_Tests/senior_living_dm")
artifacts_path = os.path.join(base_path, "artifacts")
poppler_path = os.path.join(artifacts_path, "libraries/Release-23.08.0-0/poppler-23.08.0/Library/bin")
apps_path = os.path.join(artifacts_path, "applications")
app_pdfs_path = os.path.join(apps_path, "pdfs")
apps_pngs_dir_path = os.path.join(artifacts_path, "applications/pngs")
perplexity_ocr_txts_path = os.path.join(apps_path, "perplexity_ocr")

## MRE Doc Analysis

### Step 1: Convert document to Img

In [10]:
example_pdf_path = os.path.join(app_pdfs_path, "crc_app.pdf")
example_pdf_pages = convert_from_path(pdf_path=example_pdf_path, poppler_path=poppler_path)
example_pdf_name = os.path.splitext(os.path.basename(example_pdf_path))[0]
c=1
for page in example_pdf_pages:
    os.makedirs(apps_pngs_dir_path, exist_ok=True)
    img_name = f"{example_pdf_name}_{c}.png"
    page.save(os.path.join(apps_pngs_dir_path, img_name), "PNG")
    c+=1

### Step 2: OCR the whole document

Used perplexity pro search to ocr each page png

Prompt used: OCR this document while retaining the structure. I want an output similar to ABBYY OCR. Return a python string.

In [33]:
def read_and_combine_text_files(directory):
    combined_text = ""

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                combined_text += file.read() + '\n'

    return combined_text

def write_combined_to_txt(output_filepath, combined_text):
    with open(output_filepath, 'w') as output_file:
        output_file.write(combined_text)

In [34]:
input_directory = perplexity_ocr_txts_path
output_txt_file = os.path.join(perplexity_ocr_txts_path, "mre_app.txt")

combined_text_data = read_and_combine_text_files(input_directory)

write_combined_to_txt(output_txt_file, combined_text_data)

print(f"Combined text has been written to {output_txt_file}.")

Combined text has been written to C:\Users\shaur\Desktop\Learnings\KG_Tests\senior_living_dm\artifacts\applications\perplexity_ocr\mre_app.txt.


In [39]:
def extract_sections(text, section_headers):
    sections_dict = {}
    # Create a regex pattern to match section headers
    pattern = '|'.join([re.escape(header) for header in section_headers])
    regex = rf"(?P<header>{pattern})\s*(?P<content>(?:(?!{pattern}).)*)"
    
    matches = re.finditer(regex, text, re.DOTALL)
    
    for match in matches:
        header = match.group('header').strip()
        content = match.group('content').strip()
        sections_dict[header] = content

    return sections_dict

# Read the text content
mre_ocr_app_path = os.path.join(perplexity_ocr_txts_path, "mre_app.txt")
with open(mre_ocr_app_path, 'r') as f:
    mre_ocr_app = f.read()

# List of section headers
mre_sections_list = [
    'Applicant Information',
    'Facility Information',
    'Description of Services',
    'Professional Staff',
    'Risk Management',
    'Consultants/Independent Contractors',
    'Property/Life Safety Information',
    'Commercial Automobile'
]

# Extract sections into a dictionary
sections_dict = extract_sections(mre_ocr_app, mre_sections_list)

### Step 3: Distribution of Information to correct entities

In [46]:
def divide_sections(text, section_headers, sections_dict):
    # Create a regex pattern to match section headers
    pattern = '|'.join([re.escape(header) for header in section_headers])
    regex = rf"(?P<header>{pattern})\s*(?P<content>(?:(?!{pattern}).)*)"
    
    matches = re.finditer(regex, text, re.DOTALL)
    
    for match in matches:
        header = match.group('header').strip()
        content = match.group('content').strip()
        
        # Append to the existing value in the dictionary if the key exists
        if header in sections_dict:
            sections_dict[header] += f"\n {content}"
        else:
            sections_dict[header] = content

    return sections_dict


In [47]:
entities_list = ['1. Facility', '2. Residents', '3. Claims history', '4. Licensing and Ceritfication',
'5. Risk Management', '6. Contracted Services', '7. Physical Premises and Security', '8. Corporate Structure', 
'9. Bed Census and Occupancy', '10. Coverage', '11. Services', '12. Incidents', '13. Staffing']

sections_div_dict = {}

for _, v in sections_dict.items():
    sections_div_txt = call_openai_for_entity_division(v)
    entity_div_text_dic = divide_sections(sections_div_txt, entities_list, sections_div_dict)
    

In [50]:
for k,v in entity_div_text_dic.items():
    print(k)
    print(v)
    print("*"*50)
    print()

8. Corporate Structure
Named Insured: Senior Living LLC
    Mailing Address: 1234 Northside Blvd
    City: Richardson
    State: TX
    Zip: 75080
    Tax Status: For-Profit
    Religious Affiliated: No
    Corporate Structure: LLC
    Total number of facilities owned: Not provided
    Names of other owned facilities: Not provided
    Management Company: Senior Livings
    Years with Management Company: 3
 Name of Administrator: Lily Lemon
    License Number: A321455
    State: TX
    Length of time at this facility: 3 years
    Length of time as Nursing Home Administrator (NHA): 7 years
    Full time at this facility: Yes
    Number of hours at this facility per week: 45
    Total Number of Administrators at the Facility in the last 5 years: Not provided
 (continued)
    Name of Medical Director: Hope Dsouza
    License Number: A22496
    State: TX
    Length of time as Medical Director: 13 years
    Medical Specialty: Neurology
    Full time at this facility: Not specified
    Part t